In [18]:
    import IPython, sys, pyld, types, inspect, ipywidgets, pathlib, importlib_resources, abc, itertools, urllib, importnb,contextlib, collections, requests_file, requests_cache, fnmatch, traitlets, mimetypes, operator, importlib, requests, requests_cache, urllib, ast, abc,  itertools, mimetypes, io, functools, typing
    shell = get_ipython()
    CTX, ANNO = '@context __annotations__'.split()
    __all__ = 'context', 'app'

In [20]:
    with __import__('importnb').Notebook():
        try: from . import adapters, formatters
        except: import adapters, formatters

In [21]:
    def expand(str, ctx):
        object = pyld.jsonld.expand({str: "", "@context": ctx})
        if object: return next(iter(object[0]))
        return str

In [22]:
    def treat_path(object):
        try:
            path = pathlib.Path(object)
            if path.exists(): return str(path.absolute())
        except: ...
        return object


In [23]:
    class AnnotationFormatter(IPython.core.formatters.BaseFormatter, requests.Session):
        mimetypes = traitlets.Instance(mimetypes.MimeTypes)
        
        @traitlets.default('mimetypes')
        def _default_mimetypes(Mime): 
            mimetypes = __import__('mimetypes').MimeTypes()
            try:
                with importlib_resources.path('annodize', 'base.types') as path: mimetypes.read(path)
            except:
                if __name__ == '__main__': mimetypes.read('base.types')
            return mimetypes
        
        adapter_list = traitlets.List()
        @traitlets.default('adapter_list')
        def _default_adapters(AnnotationFormatter, *object):
            http = adapters.HttpCache('local_cache')
            for adapter in [adapters.PythonAdapter(), adapters.Noop(), adapters.FileAdapter(), http, adapters.ContentsAdapter()]:
                for protocol in adapter.protocol.split(): object += [protocol, adapter],
            return list(object)
                    
        def __init__(AnnotationFormatter, *args, **kwargs):
            IPython.core.formatters.BaseFormatter.__init__(AnnotationFormatter, **kwargs), requests.Session.__init__(AnnotationFormatter)
            for key, value in AnnotationFormatter.adapter_list: AnnotationFormatter.mount(key, value)
            AnnotationFormatter.hooks['response'].append(AnnotationFormatter.post_response)
            
        def post_response(AnnotationFormatter, response, *x, **kwargs):
            response.headers['Content-Type'] =  AnnotationFormatter.mimetypes.guess_type(response.url)[0] or response.headers.get('Content-Type', '').partition(';')[0]
            try: response.callable= AnnotationFormatter.wrapped_request_response(AnnotationFormatter.lookup(response.headers['Content-Type']))
            except KeyError: ...
            return response            
        
        def __call__(AnnotationFormatter, object, ctx=None, name=None):
            return AnnotationFormatter.get(expand(treat_path(object), ctx or AnnotationFormatter.parent.user_ns.get(ANNO, {}))
                                          ) if isinstance(object, str) else object
        
        def lookup(AnnotationFormatter, object):
            obj_id = id(object)
            if obj_id in AnnotationFormatter.singleton_printers: return AnnotationFormatter.singleton_printers[obj_id]
            if not isinstance(object, str): object = IPython.core.formatters._get_type(object)
            return AnnotationFormatter.lookup_by_type(object)
    
        def lookup_by_type(AnnotationFormatter, type):
            if isinstance(type, str) and (type, None) in AnnotationFormatter.deferred_printers: return AnnotationFormatter.deferred_printers[(type, None)]
            return super().lookup_by_type(type)
            
        def for_type_by_ext(AnnotationFormatter, ext, func=None):
            return AnnotationFormatter.for_type_by_name(*AnnotationFormatter.mimetypes.guess_type(F"foo.{ext}"), func)

        @staticmethod
        def wrapped_request_response(callable):
            @functools.wraps(callable)
            def call(response):
                if len(inspect.signature(callable).parameters) == 1: return callable(response.request)
                return callable(response.request, response)
            return call

In [24]:
    def show_anno_response(response):
        callable = getattr(response, 'callable', None)
        response.object = getattr(response, 'object', None)
        if object is not None: response.object = callable(response) if callable else response        
        if response.status_code == requests.codes.ok:                    
            if isinstance(response.object, type(response)):
                try: return IPython.get_ipython().display_formatter.format(IPython.display.JSON(response.json())) 
                except: ...
            else: return IPython.get_ipython().display_formatter.format(response.object)                 
        else: return IPython.display.Markdown(response.url)

In [25]:
    def get_name(str):
        if '://' in str: return urllib.parse.parse_qs(urllib.parse.urlparse(str).query)['name'][0]
        return str

In [26]:
        def annotate_annotations(shell, object=None, state_manager=False):
            if hasattr(object, 'app'): 
                IPython.display.display(object.app)
                return
            if object is None: object = shell.user_module
            if not hasattr(object, ANNO): return
            shell_context = dict(shell.user_ns.get(ANNO, {})).get(CTX, {})
            different = {}
            annotations = dict(getattr(object, ANNO, {}))
            context = {**shell_context, **annotations.pop(CTX, {})}
            for key, value in annotations.items():
                if isinstance(value, str):
                    annotations[key] = treat_path(value)
                    context[key] = {"@type": "@vocab", "@id": F"""file://{shell.kernel.parent.connection_file}?name={key}"""}                                    
                else:
                    different[key] = value
                    if state_manager:
                        state_manager.add_traits(**{key: value})
                        widget = ipywidgets.interactive.widget_from_abbrev(value)
                        if not widget:
                            widget = ValueOutput(value=value)
                        widget.description = key
                        if hasattr(widget, 'value'):
                            state_manager.link((widget, 'value'), key)                        
                        IPython.display.display(widget)
            for key in different: 
                if key in annotations: annotations.pop(key)
                    
            expand = pyld.jsonld.expand({CTX: context, **annotations})
            if expand:
                for key, value in expand[0].items():
                    key, value = get_name(key), value[0]['@id']
                    IPython.display.display(IPython.display.Markdown(F"#### [{key}]({value})"), 
                                            shell.annotation_formatter(value, ctx=context))
            if different:
                for key, value in different.items():
                    IPython.display.display(shell.annotation_formatter(value, ctx=context))

In [27]:
    class ValueOutput(ipywidgets.Output):
        description = traitlets.Unicode()
        value = traitlets.Any()
        
        @traitlets.observe('value')
        def _observe_value(ValueOutput, change):
            with ValueOutput:
                IPython.display.display(change['new'])

In [28]:
    shell.add_traits(annotation_formatter=traitlets.Instance(AnnotationFormatter, tuple(), {'parent': shell,}, allow_none=True))
    formatters.load_ipython_extension(shell)

In [29]:
    def load_ipython_extension(shell):
        
        shell.display_formatter.mimebundle_formatter.for_type(types.ModuleType, functools.partial(annotate_annotations, shell))   
        shell.display_formatter.mimebundle_formatter.for_type(type, functools.partial(annotate_annotations, shell))           
        shell.display_formatter.mimebundle_formatter.for_type(requests.Response, show_anno_response)   

In [30]:
    def unload_ipython_extension(shell):
        null = lambda x: None
        shell.display_formatter.mimebundle_formatter.for_type(types.ModuleType, null)   
        shell.display_formatter.mimebundle_formatter.for_type(type, null)           
        shell.display_formatter.mimebundle_formatter.for_type(requests.Response, null)   

In [31]:
    def context(*files, **context):
        """A function or class decorator to add a context to the object __annotations__.
        
        This function can be used on modules to modify context.
        """
        def wrap(callable=None):
            nonlocal context
            if files:
                for name in files:
                    with open(name) as file:
                        if name.endswith(('yaml', 'yml')):
                            try: import ruamel.yaml as yaml
                            except ModuleNotFoundError: import yaml
                            context.update(yaml.safe_load(file))
                        else: context.update(__import__('json').load(file))
            if callable is None: return context
            annotations = getattr(callable, ANNO, {})
            context = {**annotations.pop(CTX, {}), **context}
            annotations[CTX] = context
            return callable
        return wrap
        

In [32]:
    class interact(traitlets.HasTraits):
        """
    >>> @interact
    ... class X:
    ...     x: 10
    >>> X.app
    <traitlets.traitlets.app...>
    >>> X.app.widgets
    {'x': IntSlider(value=10, description='x', max=30, min=-10)}
    
    """
        
        def __new__(cls, object):
            annotations = getattr(object, '__annotations__', {})
            object.app = traitlets.HasTraits.__new__(interact)
            object.app.__init__(parent=object, shell=IPython.get_ipython())
            for key, value in annotations.items():
                if key == '@context': continue
                if not key in object.app.traits():
                    object.app.add_traits(**{key: traitlets.Any})
                    object.app.observe(object.app.update_global, key)
                setattr(object.app, key, getattr(object, key, value))
                value = getattr(object.app, key)
                widget = None
                
                if isinstance(value, str):
                    value = shell.annotation_formatter(value, {
                        **getattr(shell.user_module, '__annotations__', {}).get('@context', {}),
                        **annotations.get('@context', {})})
                    if hasattr(value, 'callable'):
                        value = value.callable(value)                    
                    widget = ValueOutput()
                    widget.value = value
                    with widget:
                        IPython.display.display(value)
                else:
                    widget = ipywidgets.interactive.widget_from_abbrev(value)
                    if widget is None: widget = ValueOutput(value=value)
                    widget.description = key
                    object.app.link((widget, 'value'), key)  
                object.app.widgets[key] = widget
                
            object.app.__all__ = tuple(annotations)
            return object
                
        def update_global(State, change):
            setattr(State.parent, change['name'], change['new'])                
        
        shell = traitlets.Any()
        parent = traitlets.Any()
        widgets = traitlets.Dict()
        
        def __init__(State, **kwargs):
            super().__init__(**kwargs)
            if State.parent is None:
                State.parent = IPython.get_ipython().user_module
                State.parent.events.register('post_execute', State.post_execute)
            
        def post_execute(State):
            traits = State.traits()
            for key in State.parent.user_ns:
                if key in traits: setattr(State, key, State.parent.user_ns[key])
            
        def add_traits(State, **traits):
            current = State.traits()
            for key in traits:
                if key not in current:
                    value = traits[key]
                    if isinstance(value, traitlets.HasDescriptors): super().add_traits(**{key: value})
                    else: super().add_traits(**{key: traitlets.Any()})
                    State.observe(State.update_global, key)
                    setattr(State, key, getattr(value, 'default_value', value))
                    
            
        def dlink(State, object, name, callable=None):
            State.add_traits(**{name: getattr(*object)})
            traitlets.dlink(object, (State, name), callable)
            return object
        
        def link(State, object, name):
            State.add_traits(**{name: getattr(*object)})
            traitlets.link(object, (State, name))
            return object
        
        def _ipython_display_(State): IPython.display.display(*State.widgets.values())